In [3]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

#### Ler tabela do Google Big Query

In [7]:
# Create the connection with Google Cloud
def create_connection():
        # Set the credentials and create the connection
        key_path = "../usp-mba-dsa-tcc-4277103d9155.json"

        credentials = service_account.Credentials.from_service_account_file(
            key_path,
            scopes=["https://www.googleapis.com/auth/bigquery"]
        )

        client = bigquery.Client(
            credentials=credentials,
            project=credentials.project_id,
        )
        return client

client = create_connection()

In [11]:
# Perform a query.
QUERY = (
    'SELECT * FROM `usp-mba-dsa-tcc.ecommerce_offers.vw_dim_offers`'
)
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

df = rows.to_dataframe()

In [18]:
df.head(2)

,offer_id,Loja,Especie,Categoria,Marca,URL,SKU,Titulo,Descricao,pkg_size,title_pkg_size,pkg_value,pkg_unit,title_pkg_value,title_pkg_unit
0,b'\xd5\x87\xd5\xab\xce\xa3\xf7Fw\x0e\x12w4\xdd...,Cobasi,Cachorro,Racao Umida,Origem Natural,https://www.cobasi.com.br/racao-umida-para-cae...,31000322,Ração Úmida para Cães Adultos Joy Origem Natur...,Ração Úmida para Cães Adultos Joy Origem Natur...,300g,300g,300,g,300,g
1,b'%\xe19\xcd\xea\xae\xd9\x98\x82n\xa6\xf4M\x9f...,Cobasi,Cachorro,Racao Umida,Origem Natural,https://www.cobasi.com.br/racao-umida-para-cae...,31000330,Ração Úmida para Cães Filhotes Joy Origem Nature,Ração Úmida para Cães Filhotes Joy Origem Natu...,300g,300g,300,g,300,g


In [79]:
# Creating a copy of the dataframe and editing
offers=df
offers["Oferta"] = offers['Especie'] + " " + offers['Categoria'] + " " + offers['Marca'] + " " + offers['pkg_size'] + " " + offers['Descricao']
offers = offers[['Loja', 'offer_id', 'URL', 'Oferta']]

# Remove duplicate words inside each cell
from collections import OrderedDict
offers['Oferta'] = offers['Oferta'].astype(str).str.split().apply(lambda x: ' '.join(OrderedDict.fromkeys(x).keys()))

C:\Users\Daniel\AppData\Local\Temp\ipykernel_7792\1758800979.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offers['Oferta'] = offers['Oferta'].astype(str).str.split().apply(lambda x: ' '.join(OrderedDict.fromkeys(x).keys()))


In [80]:
offers.head(2)

,Loja,offer_id,URL,Oferta
0,Cobasi,b'\xd5\x87\xd5\xab\xce\xa3\xf7Fw\x0e\x12w4\xdd...,https://www.cobasi.com.br/racao-umida-para-cae...,Cachorro Racao Umida Origem Natural 300g Ração...
1,Cobasi,b'%\xe19\xcd\xea\xae\xd9\x98\x82n\xa6\xf4M\x9f...,https://www.cobasi.com.br/racao-umida-para-cae...,Cachorro Racao Umida Origem Natural 300g Ração...


In [84]:
print(offers["Oferta"][0])

Cachorro Racao Umida Origem Natural 300g Ração Úmida para Cães Adultos Joy Nature Hipercalórica 300 g


#### Import Language Model

In [19]:
# https://spacy.io/models/pt#pt_core_news_lg
import spacy
nlp = spacy.load("pt_core_news_lg")

In [34]:
w1 = "banana"
w2 = "mamão"

# Transform into spacy object
w1 = nlp.vocab[w1]
w2 = nlp.vocab[w2]

w1.similarity(w2)

0.6894072890281677

In [50]:
#Hx/o//XEL8KnXyhZfnMRDK55G+AoABfz3kKwGJLKAx8=
offer1 = nlp("Ração Naturalis Lifebites Cães Filhotes Porte Pequeno Peru, Frango, Legumes e Frutas 1kg")

#UTZQ/dqXqO94qscRScp/6ehZY6hZmnWKmeSatqM7QVw=
offer2 = nlp("Ração Seca Total Naturalis Peru, Frango e Frutas para Cães Adultos Porte Pequeno - 1 Kg")

offer1.similarity(offer2)

0.8842713063123244

In [51]:
#N954En5srMC24fi4K/3au+jmzCuc6guQx7x1IYwDvM0=
offer2 = nlp("Ração Naturalis Lifebites Cães Sênior Porte Pequeno Peru, Frango, Legumes e Frutas 1kg")
offer1.similarity(offer2)

0.992868997469065

In [52]:
#fTRC8JQlzMAhL3qgENIRP0HJz2x+QbmoWUqfvyX1cfs=
offer2 = nlp("Ração Úmida Fórmula Natural Vet Care Recuperação para Cães e Gatos 270 g")
offer1.similarity(offer2)

0.7457927405154836

In [53]:
#qB1kHKxZOmz4J6A/OfWybRC260BtjXb0vm/9q4SlM1I=
offer2 = nlp("Ração Naturalis Lifebites Cães Adultos Porte Médio e Grande Peru, Frango, Legumes e Frutas 2,5kg")
offer1.similarity(offer2)

0.9135871309459083

In [47]:
#yfkeevB9AQTAnuC97yDgi7DWo8H/anZQ57/QHngrKLY=
offer2 = nlp("Ração Naturalis Lifebites Gatos Adultos Peru, Frango e Frutas 1,5kg")
offer1.similarity(offer2)

0.7833030219414631

In [54]:
#KlqW6FrilTY99taOFOyzXbwjXgR/2SkyRzNaYUlFIHA=
offer2 = nlp("Ração Naturalis Cães Filhotes Pequeno Porte Peru, Frango e Frutas 1 kg")
offer1.similarity(offer2)

0.9880225975281419

In [89]:
offer1 = nlp(offers["Oferta"][0])
offer2 = nlp(offers["Oferta"][3])

print(f"{offer1}\n{offer2}\n{offer1.similarity(offer2)}")


Cachorro Racao Umida Origem Natural 300g Ração Úmida para Cães Adultos Joy Nature Hipercalórica 300 g
Cachorro Racao Umida Joy 280g Ração Úmida Patê para Cães Adultos Carne e Vegetais 280 g
0.9347718981637252
